In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers trl peft accelerate bitsandbytes

In [ ]:
best_params = {
    'learning_rate': 0.0007804852426265795,
    'batch_size': 1,
    'warmup_steps': 3,
    'max_steps': 109,
    'weight_decay': 0.00031584871539798177,
    'dropout_rate': 0.4251796832408427,
}

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("mbpp", split = "train")
validate_dataset = load_dataset("mbpp", split = "validation")

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = best_params['dropout_rate'], # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.4251796832408427.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
import pandas as pd

# Define your custom prompt template
alpaca_prompt = """Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
{}

### Test Case:
{}

### Response:
{}"""

# Define the EOS_TOKEN (replace 'tokenizer' with your actual tokenizer instance)
EOS_TOKEN = '<eos>'  # Example token, replace with tokenizer.eos_token

# Function to format the dataset examples
def formatting_prompts_func(examples):
    instructions = examples["text"]
    test_cases = examples["test_list"] #give all test case
    outputs = examples["code"]
    texts = []

    for instruction , test_case, output in zip(instructions, test_cases, outputs):

        # Format the text using the template and add EOS_TOKEN
        text = alpaca_prompt.format(instruction, test_case, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

# Apply the formatting function to the dataset
format_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)
format_validate_dataset = validate_dataset.map(formatting_prompts_func, batched=True, remove_columns=validate_dataset.column_names)

In [ ]:
print(format_dataset['text'][0])

Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
Write a function to find the longest chain which can be formed from the given set of pairs.

### Test Case:
['assert max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) == 3', 'assert max_chain_length([Pair(1, 2), Pair(3, 4),Pair(5, 6), Pair(7, 8)], 4) == 4', 'assert max_chain_length([Pair(19, 10), Pair(11, 12),Pair(13, 14), Pair(15, 16), Pair(31, 54)], 5) == 5']

### Response:
class Pair(object): 
	def __init__(self, a, b): 
		self.a = a 
		self.b = b 
def max_chain_length(arr, n): 
	max = 0
	mcl = [1 for i in range(n)] 
	for i in range(1, n): 
		for j in range(0, i): 
			if (arr[i].a > arr[j].b and
				mcl[i] < mcl[j] + 1): 
				mcl[i] = mcl[j] + 1
	for i in range(n): 
		if (max < mcl[i]): 
			max = mcl[i] 
	return max<eos>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Define TrainingArguments with the best hyperparameters
training_args = TrainingArguments(
    per_device_train_batch_size=best_params['batch_size'],
    gradient_accumulation_steps=4,
    warmup_steps=best_params['warmup_steps'],
    max_steps=best_params['max_steps'],
    learning_rate=best_params['learning_rate'],
    fp16=True,
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=best_params['weight_decay'],
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    evaluation_strategy="steps",
    eval_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Initialize the trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=format_dataset,
    eval_dataset=format_validate_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/374 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/90 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 374 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 109
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
10,1.081900,0.578750
20,0.568300,0.545433
30,0.564900,0.530083
40,0.501300,0.526712
50,0.554200,0.520942
60,0.515700,0.512320
70,0.558700,0.505990
80,0.472800,0.503826
90,0.519200,0.497919
100,0.434500,0.496085


In [ ]:
from datasets import load_dataset
test_dataset = load_dataset("mbpp", split = "test")

In [ ]:
instruction = test_dataset['text'][0]
test_case = test_dataset['test_list'][0][0]
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        test_case,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

<s> Write a Python code to solve the given problem, and the code should be able to run the test case.

### Instruction:
Write a python function to remove first and last occurrence of a given character from the string.

### Test Case:
assert remove_Occ("hello","l") == "heo"

### Response:
def remove_Occ(str,char):
  str = str.replace(char, "")
  str = str.replace(char, "")
  return (str) <eos>

<eos>
<|endoftext|>


In [ ]:
import re
# Define the regex pattern to extract code between "### Response:" and the first <eos>
pattern = re.compile(r"### Response:\s*(.*?)<eos>", re.DOTALL)

# Find all matches of the pattern in the decoded output
matches = pattern.findall(tokenizer.batch_decode(outputs)[0])

# Extract the first match which is the desired code block
if matches:
    extracted_code = matches[0].strip()
    print("Extracted Code:\n", extracted_code)
else:
    print("No code block found.")

# Optional: further processing of the extracted code if needed

Extracted Code:
 def remove_Occ(str,char):
  str = str.replace(char, "")
  str = str.replace(char, "")
  return (str)


In [ ]:
import re
import json

# Define the regex pattern to extract code between "### Response:" and the first <eos>
pattern = re.compile(r"### Response:\s*(.*?)<eos>", re.DOTALL)

# Define the checkpoint frequency
checkpoint_frequency = 10

# Load previous results if a checkpoint file exists
file = "results.json"
try:
    with open(file, "r") as f:
        results = json.load(f)
except FileNotFoundError:
    results = []

# Determine the starting index based on the length of the existing results
start_index = len(results)

for i in range(start_index, len(test_dataset['text'])):
    instruction = test_dataset['text'][i]
    test_case = test_dataset['test_list'][i][0]
    test_cases = test_dataset['test_list'][i]
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,  # instruction
                test_case,
                "",  # output - leave this blank for generation!
            )
        ], return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=2048, use_cache=True,)

    generated_text = tokenizer.batch_decode(outputs)[0]

    # Find all matches of the pattern in the generated output
    matches = pattern.findall(generated_text)

    # Extract the first match which is the desired code block
    if matches:
        extracted_code = matches[0].strip()
    else:
        extracted_code = "No code block found."

    # Store the test case and extracted code as a pair
    results.append((test_cases, extracted_code))

    # Save the checkpoint every 10 iterations
    if (i + 1) % checkpoint_frequency == 0:
        with open(file, "w") as f:
            json.dump(results, f, indent=4)
        print(f"Checkpoint saved at index {i + 1}")

# Final save after the loop ends
with open(file, "w") as f:
    json.dump(results, f, indent=4)
print("Final checkpoint saved.")

# Display or save the results
for test_case, code in results:
    print(f"Test Case: {test_case[0]}\nExtracted Code:\n{code[0]}\n")



Checkpoint saved at index 10
Checkpoint saved at index 20
Checkpoint saved at index 30
Checkpoint saved at index 40
Checkpoint saved at index 50
Checkpoint saved at index 60
Checkpoint saved at index 70
Checkpoint saved at index 80
Checkpoint saved at index 90
Checkpoint saved at index 100


In [ ]:
print("def sort_matrix(matrix):\r\n\trow_sum = []\r\n\tfor index, row in enumerate(matrix):\r\n\t\trow_sum.append(sum(row))\r\n\tmatrix.sort(key = lambda x: row_sum[matrix.index(x)])\r\n\trow_sum.sort()# to get the index of the next smallest sum\r\n\tfor index, row in enumerate(matrix):\r\n\t\tif row_sum[index] == row_sum[index - 1]:\r\n\t\t\t\tmatrix.remove(matrix[index - 1])\r\n\t\t\t\tmatrix.insert(index - 1, matrix[-1])\r\n\t\t\t\tmatrix.remove(matrix[-1])\r\n\treturn matrix")

def sort_matrix(matrix):
	row_sum = []
	for index, row in enumerate(matrix):
		row_sum.append(sum(row))
	matrix.sort(key = lambda x: row_sum[matrix.index(x)])
	row_sum.sort()# to get the index of the next smallest sum
	for index, row in enumerate(matrix):
		if row_sum[index] == row_sum[index - 1]:
				matrix.remove(matrix[index - 1])
				matrix.insert(index - 1, matrix[-1])
				matrix.remove(matrix[-1])
	return matrix
